In [89]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [90]:
train_df = pd.read_csv('input/train.csv')
# train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test.csv')

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [91]:
n_splits = 10
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [92]:
# cv: 0.89951
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4 
        }

'''
# cv: 0.90775 (full)
# lb: 0.891 with fold_n=2 => overfit?
params = {
         'objective': 'binary',
         'max_depth': 127,
         'metric': 'auc',
         'boosting':'gbdt',
         }
'''

# 1st round looks good: cv 0.900043
# try larget max_bin?
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 1000,
         'objective': 'binary',
         'max_depth': -1,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4,
         'max_bin': 511,
        }

In [93]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=200000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits

Fold 0 started at Fri Mar  1 15:22:06 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.826497	valid_1's auc: 0.815838
[600]	training's auc: 0.860757	valid_1's auc: 0.847247
[900]	training's auc: 0.879147	valid_1's auc: 0.862331
[1200]	training's auc: 0.890363	valid_1's auc: 0.871118
[1500]	training's auc: 0.898484	valid_1's auc: 0.877528
[1800]	training's auc: 0.904564	valid_1's auc: 0.881739
[2100]	training's auc: 0.909327	valid_1's auc: 0.885046
[2400]	training's auc: 0.913299	valid_1's auc: 0.887495
[2700]	training's auc: 0.916567	valid_1's auc: 0.889461
[3000]	training's auc: 0.919369	valid_1's auc: 0.89133
[3300]	training's auc: 0.921728	valid_1's auc: 0.892845
[3600]	training's auc: 0.923778	valid_1's auc: 0.893837
[3900]	training's auc: 0.925537	valid_1's auc: 0.894773
[4200]	training's auc: 0.927104	valid_1's auc: 0.89563
[4500]	training's auc: 0.928549	valid_1's auc: 0.896428
[4800]	training's auc: 0.929919	valid_1's auc: 0.896978
[51

Fold 5 started at Fri Mar  1 16:07:19 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.826565	valid_1's auc: 0.815318
[600]	training's auc: 0.860974	valid_1's auc: 0.846394
[900]	training's auc: 0.878686	valid_1's auc: 0.861706
[1200]	training's auc: 0.890012	valid_1's auc: 0.871455
[1500]	training's auc: 0.89812	valid_1's auc: 0.878292
[1800]	training's auc: 0.904351	valid_1's auc: 0.883493
[2100]	training's auc: 0.909021	valid_1's auc: 0.887128
[2400]	training's auc: 0.912848	valid_1's auc: 0.889911
[2700]	training's auc: 0.916148	valid_1's auc: 0.892095
[3000]	training's auc: 0.918951	valid_1's auc: 0.894249
[3300]	training's auc: 0.921345	valid_1's auc: 0.895887
[3600]	training's auc: 0.923394	valid_1's auc: 0.897021
[3900]	training's auc: 0.925062	valid_1's auc: 0.898055
[4200]	training's auc: 0.926629	valid_1's auc: 0.899073
[4500]	training's auc: 0.928035	valid_1's auc: 0.899569
[4800]	training's auc: 0.929404	valid_1's auc: 0.900204
[5

In [94]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)